In [42]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader, random_split
import torchmetrics.functional as metrics
from torchvision.datasets import ImageFolder
from torchvision.transforms import transforms
from bs4 import BeautifulSoup
import os
from torchinfo import summary
from torchvision.models import resnet18

In [43]:
# 군용 항공기 종류 식별자
AIRCRAFT_TYPE = {
    'A':'Attater',
    'B':'Bomber',
    'C':'Cargo',
    'F':'Fighter',
    'AH':'Attack Helicopter',
    'CH':'Cargo Helicopter',
    'UH':'Utility Helicopter'
}

In [44]:
# 군용 항공기 제조국 식별자
AIRCRAFT_NATION = [
    'USA',
    'EUROPE',
    'RUSSIA',
    'CHINA'
]

In [45]:
# 군용 항공기 명명 식별자
AIRCRAFT_NAME = [
    # USA
    'A-10',
    'B-1',
    'B-2',
    'B-52',
    'C-5',
    'C-17',
    'C-130',
    'CV-22',
    'F-15',
    'F-16',
    'FA-18',
    'F-22',
    'F-35',
    'AH-1',
    'AH-64',
    'CH-46',
    'CH-47',
    'CH-53',
    'UH-60',
    # Europe
    'EF2000',
    'Rafale',
    'JAS-39',
    'AV-8',
    # Russia
    'MIG-29',
    'MIG-31',
    'MIG-35',
    'Su-30',
    'Su-34',
    'Su-57',
    'Tu-160',
    'Mi-8',
    'Mi-24',
    'Mi-28',
    'Ka-52',
    # CHINA
    'J-15',
    'J-20',
    'H-6',
    'Y-20',
    'WZ-10',
    # Unidentified
    'UNIDENTIFIED'
]

In [46]:
len(AIRCRAFT_NAME)

40

In [47]:
torch.random.manual_seed(40)

In [48]:
data_path = './data/2/'

for name in AIRCRAFT_NAME:
    os.makedirs(os.path.join(data_path, name), exist_ok=True)

In [49]:
transformer = transforms.Compose(transforms=[
    transforms.ToTensor(),
    transforms.Resize(size=[96, 96])]
)

In [50]:
root_path = './data/'

ImgDS_1 = ImageFolder(root_path+'1/', transform=transformer)
ImgDS_2 = ImageFolder(root_path+'2/', transform=transformer)
ImgDS_3 = ImageFolder(root_path+'crop/', transform=transformer)

In [51]:
for f, t in ImgDS_1:
    print(f.shape, t)
    break

torch.Size([3, 96, 96]) 0


In [52]:
set(ImgDS_1.targets)

{0, 1, 2, 3, 4}

In [53]:
set(ImgDS_2.targets)

{0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 12,
 13,
 14,
 15,
 16,
 17,
 18,
 19,
 20,
 21,
 22,
 23,
 24,
 25,
 26,
 27,
 28,
 29,
 30,
 31,
 32,
 33,
 34,
 35,
 36,
 37,
 38,
 39}

In [54]:
class CNN(nn.Module):
    def __init__(self, num_classes):
        super(CNN, self).__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(3, 128, 3, padding='same'),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(128, 256, 3, padding='same'),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            
            nn.Conv2d(256, 256, 3, padding='same'),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            
            nn.Conv2d(256, 256, 3, padding='same'),
            nn.BatchNorm2d(256),
            nn.ReLU(),
            
            nn.Conv2d(256, 128, 3, padding='same'),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            
            nn.Conv2d(128, 128, 3, padding='same'),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.AvgPool2d(kernel_size=2, stride=2),
            
            nn.Flatten(),
            nn.Linear(128*12*12, num_classes)
        )
        
    def forward(self, x):
        return self.layers(x)

In [55]:
def makeLoader(self, batch=1, sampler=None):
    train, valid, test = random_split(self, [0.7, 0.1, 0.2],
                                      torch.Generator().manual_seed(40))
    trainDL = DataLoader(train, batch_size=batch, sampler=sampler, shuffle=True)
    validDL = DataLoader(valid, batch_size=batch, sampler=sampler, shuffle=True)
    testDL = DataLoader(test, batch_size=batch, sampler=sampler, shuffle=True)
    
    return trainDL,validDL,testDL

In [56]:
train_cost_list = []
train_acc_list = []
valid_cost_list = []
valid_acc_list = []

In [57]:
def train_model(model, opt, epochs, trainDL, validDL, schd=None):
    for epoch in range(1, epochs+1):
        for f, t in trainDL:
            model.train()
            sum_acc = 0
            
            h = model(f)
            # print(f.shape, t.shape, h.shape)
            
            cost = F.cross_entropy(h, t)
            
            opt.zero_grad()
            cost.backward()
            opt.step()
            
            sum_acc += metrics.accuracy(h, t, task='multiclass', num_classes=model.layers[-1].out_features)
            print('.', end='')
            
        train_cost_list.append(cost)
        train_acc_list.append(sum_acc / len(trainDL))
        
        for f, t in validDL:
            model.eval()
            sum_acc = 0
            
            h = model(f)
            
            cost = F.cross_entropy(h, t)
            
            sum_acc += metrics.accuracy(h, t, task='multiclass', num_classes=model.layers[-1].out_features)
            
        valid_cost_list.append(cost)
        valid_acc_list.append(sum_acc / len(validDL))
        
        print(f'Epoch [{epoch:4}/{epochs:4}] ----------')
        print(f'Train cost : {train_cost_list[-1]}, Train acc : {train_acc_list[-1]}')
        print(f'Valid cost : {valid_cost_list[-1]}, Valid acc: {valid_acc_list[-1]}')
        
        if schd is not None:
            schd.step(valid_cost_list[-1])
        

In [58]:
def test_model(model, testDL):
    test_acc_list = []
    test_f1_list = []
    
    model.eval()
    for f, t in testDL:
        h = model(f)
        
        test_acc_list.append(metrics.accuracy(h, t, task='multiclass', num_classes=model.layers[-1].out_features))
        test_f1_list.append(metrics.f1_score(h, t, task='multiclass', num_classes=model.layers[-1].out_features))
        
    print(f'Average Test acc : {sum(test_acc_list) / len(testDL)}')
    print(f'Average Test f1  : {sum(test_f1_list) / len(testDL)}')

In [59]:
def exportModel(model, filename):
    pass

In [60]:
def model_predict(model, input):
    pass

In [61]:
trainDL1, validDL1, testDL1 = makeLoader(ImgDS_1, 64)
trainDL2, validDL2, testDL2 = makeLoader(ImgDS_2, 64)
trainDL3, validDL3, testDL3 = makeLoader(ImgDS_3, 64)

In [62]:
model1 = CNN(len(os.listdir('./data/1/')))

In [63]:
model1.layers[-1].out_features

5

In [64]:
model2 = CNN(len(AIRCRAFT_NAME))

In [65]:
model3 = CNN(len(os.listdir('./data/crop/')))

In [66]:
summary((model3))

Layer (type:depth-idx)                   Param #
CNN                                      --
├─Sequential: 1-1                        --
│    └─Conv2d: 2-1                       3,584
│    └─BatchNorm2d: 2-2                  256
│    └─ReLU: 2-3                         --
│    └─MaxPool2d: 2-4                    --
│    └─Conv2d: 2-5                       295,168
│    └─BatchNorm2d: 2-6                  512
│    └─ReLU: 2-7                         --
│    └─Conv2d: 2-8                       590,080
│    └─BatchNorm2d: 2-9                  512
│    └─ReLU: 2-10                        --
│    └─MaxPool2d: 2-11                   --
│    └─Conv2d: 2-12                      590,080
│    └─BatchNorm2d: 2-13                 512
│    └─ReLU: 2-14                        --
│    └─Conv2d: 2-15                      295,040
│    └─BatchNorm2d: 2-16                 256
│    └─ReLU: 2-17                        --
│    └─Conv2d: 2-18                      147,584
│    └─BatchNorm2d: 2-19              

In [67]:
opt1 = optim.Adam(model1.parameters())

In [68]:
opt2 = optim.Adam(model2.parameters())

In [69]:
opt3 = optim.Adam(model3.parameters())

In [70]:
schd1 = optim.lr_scheduler.ReduceLROnPlateau(opt1)

In [71]:
schd2 = optim.lr_scheduler.ReduceLROnPlateau(opt2)

In [72]:
schd3 = optim.lr_scheduler.ReduceLROnPlateau(opt3)

In [ ]:
train_model(model3, opt3, 50, trainDL3, validDL3, schd3)

.....................................................................................................................................................................................................................................................Epoch [   1/  50] ----------
Train cost : 3.6720738410949707, Train acc : 0.00022675737272948027
Valid cost : 3.6406149864196777, Valid acc: 0.002005012473091483
.....................................................................................................................................................................................................................................................Epoch [   2/  50] ----------
Train cost : 3.4335711002349854, Train acc : 0.0009070294909179211
Valid cost : 3.0784494876861572, Valid acc: 0.00501253129914403
............................................................................................................................................................................................